# Grid operation-based outage maintenance optimization challange
ROADEF/EURO and RTE, the electricity transmission system operator of France, issued in 2020 a public challange to optimise the grid operation-based outage maintenance planning. They provided either the problem definition and some data sets representing different problem instances. This is a perfect situation to test also Quantum Optimisation Algortihms on a real use case publicly discussed.

## Problem definition (short summary form [challenge specifications](https://github.com/rte-france/challenge-roadef-2020/raw/master/Challenge_Subject.pdf))

> Guaranteeing both electricity delivery and supply is one of the most important mission of a transmission system operator such as RTE. But such an objective can be carried out only if the grid is correctly maintained. In particular, some maintenance operations on the overhead power lines are live-line works while others require to shut the power down. When this happens, both electricity delivery and supply have to be guaranteed, meaning that maintenance operations have to be planned carefully. When there is not any maintenance operation, the network is resilient enough to endure an unexpected contingency. However, if several breakdowns occur, the grid might face major blackouts. In this context, planned outages due to maintenance work have to be scheduled with extreme caution.
>
> \[...\]
>
> RTE decided to implement a three-step approach. First, risk values corresponding to different future scenarios are computed. Second, these computed values are included in several optimisation approaches in order to find a good schedule. Eventually, a third step validates the obtained planning. This challenge focuses on the second step of this approach: given the risk values, the goal is to find an optimal planning regarding a risk-based objective. Moreover, this planning must be consistent with all job-related restrictions such as resource constraints.

The planning is organized in time steps denoted by $ t \in H=[1, \dots, T] \subset \mathbb{N} $. A schedule is composed by the list of all given interventions $i \in I$ with their starting time step $s_{i}$. Any task should be completed by the end of the planning time interval.

To carry out the different interventions (task), resources with differenct specific skills $ c \in C $ are required, with some constarints:
- the available resources are always limited and therefore there is a maximum value $ rmax^{c,t} $ for resources with skill $c$ that cannot be exceeded at time $t$
- for operational reasons, there is also a lower bound $ rmin^{c,t} $  on the resources consumption in order to prevent workforce from not being used at all

Any intervention $i$ has:
- a duration $ d_{i,s} $ (expressed in time steps) that depends on the starting time step $s$ of the intervention $i$
- a resource workload $ wl^{c,t}_{i,s}$ that represents the fraction number of resources with skill $c$ needed at time step $t$ if the intervention $i$ starts a time $s$
- a risk value $ rk^{r,t}_{i,s}$ that represents the risk at time $t$ in the risk scenario $r$ related to the intervention $i$ starting at time $s$; a different number $ R_{t}$ of risk scenarios may be defined for any time step $t$

There are some couples $(i,j)$ of interventions that cannot be performed jointly in specific time periods; a list $E$ of these exclusions $(i,j,t) \in I \times I \times H$ is also provided.

The main goal is to minimize the avarage risk of the schedule, defined as $ \frac{1}{T}\sum_{t}{\frac{1}{R_{t}}\sum_{r}{risk^{r,t}}} $ where $ risk^{r,t} $ is the total risk value related to the running interventions at time $t$ in the risk scenario $r$.

## Problem instances

Many datasets have been provided by RTE in the form of JSON files that includes:
- Resources with upper and lower bounds
- Sesons with time steps
- Interventions
  - Duration and latest valid starting time step 
  - Workload for required resources
  - Risk values
- Exclusions

In [27]:
# Import input data file

import logging
import json
import numpy as np
import sys

input_file =  '.\data\C_10.json' # 'example1.json'  #
output_file = 'output.txt'

# Set logger
log = logging.getLogger()
log.setLevel(logging.INFO)
try: consoleHandler
except NameError: 
    consoleHandler = logging.StreamHandler(sys.stdout)
    log.addHandler(consoleHandler)

# Open and parse JSON file
log.info(f'Reading file \'{input_file}\'')
f = open(input_file, 'r')
data = json.load(f)
f.close()

Reading file '.\data\C_10.json'


In [28]:
# Problem setting

log.info(f'Problem setting ...')

# Resources (C)
res = {} # Resouce names --> c = 0..(C-1)
rmax_c_t = [] # Max usage for resource c
rmin_c_t = [] # Min usage for resource c
for i,k in enumerate(data['Resources']):
    res[k] = i
    rmax_c_t += [data['Resources'][k]['max']]
    rmin_c_t += [data['Resources'][k]['min']]
C = len(res)

def findResourceName(id) :
    for k in res.keys() :
        if res[k] == id : 
            return k
    return None

# Calendar (T)
time = [] # Time periods t = 0..(T-1)
for k in data['Seasons'].keys():
    time += data['Seasons'][k]
time = [int(time[i]) for i in range(len(time))]
time.sort()
T = len(time)

# Interventions (I, S)
intv = [] # Interventions' names --> i = 0..(I-1)
d_i_s = [] # Duration of intervention i starting at time s
smax_i = [] # Latest starting time for intervention i

# Get only startable interventions
for i,k in enumerate(data['Interventions']):
    if int(data['Interventions'][k]['tmax']) <= T :
        intv += [k]
    else:
        log.warning(f'Intervenion {k} cannot start in the time framework')

# Get interventions data
I = len(intv)
wl_i_c_t_s = np.zeros((I, C, T, T), dtype=float) # Worklod of recource c for intervention i at time t when i started at time s
rk_r_i_t_s = [] # Risk for scenario r for intervention i at time t when i started at time s
R_t = np.zeros(T, dtype=int) # Risk scenarios at time t

for i,k in enumerate(intv):
    smax_i += [int(data['Interventions'][k]['tmax'])-1]
    d_i_s += [[int(d) for d in data['Interventions'][k]['Delta']]]
    # Workload
    for r in data['Interventions'][k]['workload'].keys():
        c = res[r]
        for t in data['Interventions'][k]['workload'][r].keys():
            for s in data['Interventions'][k]['workload'][r][t].keys():
                wl_i_c_t_s[i][c][int(t)-1][int(s)-1]=data['Interventions'][k]['workload'][r][t][s]
    # Risks
    for t in data['Interventions'][k]['risk'].keys():
        for s in data['Interventions'][k]['risk'][t].keys():
            if R_t[int(t)-1] == 0 or R_t[int(t)-1] == len(data['Interventions'][k]['risk'][t][s]) :
                R_t[int(t)-1] = len(data['Interventions'][k]['risk'][t][s])
            else :
                print(f'Warning at time {t} R_t not unique')
            for r,v in enumerate(data['Interventions'][k]['risk'][t][s]):
                if len(rk_r_i_t_s)-1 < r : 
                    rk_r_i_t_s +=[np.zeros((I, T, T), dtype=float)]
                rk_r_i_t_s[r][i][int(t)-1][int(s)-1] = v

dmax_i = [max(d_i_s[i]) for i in range(I)]

def findInterventionByName(name) :
    for i,n in enumerate(intv) :
        if n == name : 
            return i
    return -1


def riskAnalysis() :
    max_avg_risk=0
    min_avg_risk=0
    max_obj=0
    for i in range(I):
        r_s = []
        for s in range(smax_i[i]+1):
            risk = 0
            for t in range(s,min(s+d_i_s[i][s]+1, T)):
                for r in range(R_t[t]):
                    risk += rk_r_i_t_s[r][i][t][s]/R_t[t]
            r_s+=[risk]     
        max_avg_risk += max(r_s)/T
        min_avg_risk += min(r_s)/T
        max_obj += sum(r_s)
    return {"max_avg_risk": max_avg_risk, "min_avg_risk": min_avg_risk, "max_obj": max_obj}

ra = riskAnalysis()

# Exclusions
exc_e = []
for e,k in enumerate(data['Exclusions']):
    int_i = data['Exclusions'][k][0]
    int_j = data['Exclusions'][k][1]
    exc_t = data['Seasons'][data['Exclusions'][k][2]]
    if len(exc_t) > 0 :
        exc_e += [{'e':k, 'i':findInterventionByName(int_i), 'j':findInterventionByName(int_j), 't': [int(t)-1 for t in exc_t] }]
E = len(exc_e) 

log.info(f'\tTime periods: {T}')
log.info(f'\tInterventions: {I}')
log.info(f'\tAverage max duration of interventions: {round(np.mean([max(d_i_s[i]) for i in range(I)]))}')
log.info(f'\tResources: {C}')
log.info(f'\tExlusions: {E}')
log.info(f'\tMax risk scenarions per time period: {max(R_t)}')
log.info(f'\tRisk analysis: avarage risk is between {round(ra["min_avg_risk"])} and {round(ra["max_avg_risk"])}')
log.info(f'\tMax objective function value: {round(ra["max_obj"]):,}')
log.info(f'\tWorkload matrix size: {I*C*T*T:,}')
log.info(f'\tRisk matrix size: {max(R_t)*I*T*T:,}')

Problem setting ...
	Time periods: 102
	Interventions: 522
	Average max duration of interventions: 31
	Resources: 9
	Exlusions: 705
	Max risk scenarions per time period: 69
	Risk analysis: avarage risk is between 24986 and 34384
	Max objective function value: 168,423,231
	Workload matrix size: 48,877,992
	Risk matrix size: 374,731,272


## Model definition

The problem has been represented as a Constrained Quadratic Model with binary viariables:

$$ \forall i \in I, \forall s \in H_i \qquad x_{i,s}=\begin{cases} 1 & \text{Intervention } i \text{ starts at time } s \\ 0 & \text{Otherwise} \end{cases} \quad \text{where} \quad H_i = \{ s \in H \, | \, s+d_{i,s} \leq T \}  $$

The objective function is defined as:

$$  \frac{1}{T}\sum_{t}{\frac{1}{R_{t}}\sum_{r}{\sum_{i}{\sum_{s}{rk^{r,t}_{i,s}x_{i,s}}}}}  $$

The following constraints must hold:

- Every task starts exactly once 

$$ \forall i \in I \qquad \sum_{s}{x_{i,s}} = 1 $$

- Resources' usage is in the specified range

$$ \forall c \in C, \forall t \in H \qquad rmin^{c,t} \leq \sum_{i}{\sum_{s}{wl^{c,t}_{i,s} x_{i,s}}} \leq rmax^{c,t} $$ 

- The default exclusions are respected (quadratic formulation of the constraint)

$$ \forall (i,j,t) \in E \qquad \sum_{s_i \in S^t_i}{\sum_{s_j \in S^t_j}{x_{i,s_i}x_{j,s_j}}} = 0 \; \text{ where } \; S^t_i = \{ s \in H \,|\, s \leq t \leq s+d_{i,s} \} $$

- The default exclusions are respected (alternative linear formulation)

$$ \forall (i,j,t) \in E, \forall s_i \in S^t_i, \forall s_j \in S^t_j \qquad x_{i,s_i}+x_{j,s_j} \leq 1 $$

In [29]:
# Create Model

from dimod import Binary, BinaryQuadraticModel, ConstrainedQuadraticModel 

# Model
log.info(f'Creating model ...')
qm = ConstrainedQuadraticModel()

# Model variables
x_i_s = [[Binary(f'x_{i}_{s}') for s in range(smax_i[i]+1)] for i in range(I)]

log.info(f'\tVariables: {len([x_i_s[i][s] for i in range(I) for s in range(smax_i[i]+1) ])}')

Creating model ...
	Variables: 41087


In [30]:
# Constraints
LM = 1

# All interventions is executed one and only one time (4.1.2)
constraints_412 = 0
for i in range(I):
    qm.add_constraint(sum([LM*x_i_s[i][s] for s in range(0,smax_i[i]+1)]) == 1*LM, label=f'Task_{intv[i]}_starts_exactly_once', weight = None)
    constraints_412 += 1

log.info(f'\tConstraints "Interventions are scheduled once": {constraints_412}')

	Constraints "Interventions are scheduled once": 522


In [31]:
# Resource usage constraints (4.1.3)
constraints_42 = 0
LM = 1

for c in range(C):
    for t in range(T):
        cons = [x_i_s[i][s]*wl_i_c_t_s[i][c][t][s]*LM for i in range(I) for s in range(0,min(t,smax_i[i])+1) if wl_i_c_t_s[i][c][t][s]>0 ] # and s <= t and s+d_i_s[i][s]-1 >= t
        if len(cons) != 0:
            if rmin_c_t[c][t] > 0 : 
                qm.add_constraint(sum(cons) >= rmin_c_t[c][t]*LM, label=f'No_underun_for_{findResourceName(c)}_at_{time[t]}')
                constraints_42 += 1
            if sum([wl_i_c_t_s[i][c][t][s] for i in range(I) for s in range(0,min(t,smax_i[i])+1)]) > rmax_c_t[c][t] :
                qm.add_constraint(sum(cons) <= rmax_c_t[c][t]*LM, label=f'No_overrun_for_{findResourceName(c)}_at_{time[t]}')
                constraints_42 += 1
        else:
            if rmin_c_t[c][t] > 0 : print(f'WARNING: No possible workload at time {time[t]} for resource {findResourceName(c)}')

log.info(f'\tConstraints "Resource usage": {constraints_42}')  


	Constraints "Resource usage": 889


In [32]:
# Exclusions (quadratic constraint)

constraints_43 = 0
LM = 1

for e in exc_e:
    prod = []
    for t in e['t']:
        for si in range(max(0,t-dmax_i[e['i']]), min(t, smax_i[e['i']])+1):
            if si+d_i_s[e['i']][si]-1 >= t :
                for sj in range(max(0,t-dmax_i[e['j']]), min(t, smax_i[e['j']])+1):
                    if sj+d_i_s[e['j']][sj]-1 >= t :
                        if not [si, sj] in prod:
                            prod += [[si, sj]]
    cons = [LM*x_i_s[e['i']][p[0]]*x_i_s[e['j']][p[1]] for p in prod]
    qm.add_constraint(sum(cons) == 0, label=f"Exclusion_{e['e']}_{intv[e['i']]}_and_{intv[e['j']]}")
    constraints_43 += 1
    
log.info(f'\tQuadratic constraints "Exclusions": {constraints_43}') 

	Quadratic constraints "Exclusions": 705


In [33]:
# Exclusions (linear constraint)

constraints_43 = 0
LM = 1

for e in []: # not used
    prod = []
    for t in e['t']:
        for si in range(max(0,t-dmax_i[e['i']]), min(t, smax_i[e['i']])+1):
            if si+d_i_s[e['i']][si]-1 >= t :
                for sj in range(max(0,t-dmax_i[e['j']]), min(t, smax_i[e['j']])+1):
                    if sj+d_i_s[e['j']][sj]-1 >= t :
                        if not [si, sj] in prod:
                            prod += [[si, sj]]
    for p in prod:
        qm.add_constraint(LM*(x_i_s[e['i']][p[0]]+x_i_s[e['j']][p[1]]) <= LM, label=f"Exclusion_{e['e']}_{intv[e['i']]}_starts_at_{time[p[0]]}_and_{intv[e['j']]}_sj={time[p[1]]}")
        constraints_43 += 1
    
log.info(f'\tLinear constraints "Exclusions": {constraints_43}')     

	Linear constraints "Exclusions": 0


In [34]:
# Objective function

avgrisk = []
for i in range(I):
    for s in range(smax_i[i]+1):
        r_i_s = 0
        for t in range(s,min(s+d_i_s[i][s]+1, T)):
            for r in range(R_t[t]):
                r_i_s += rk_r_i_t_s[r][i][t][s]/R_t[t]
        if r_i_s > 0:
            avgrisk += [r_i_s*x_i_s[i][s]]
    
qm.set_objective(sum(avgrisk))

log.info(f'\tObjective function "Avarage risk" terms: {len(avgrisk)}')

	Objective function "Avarage risk" terms: 41087


In [35]:
from dwave.system import LeapHybridCQMSampler 

sampler = LeapHybridCQMSampler()
mtl = sampler.min_time_limit(qm)
time_limit = max(200, mtl)

log.info(f'Start sampling ... ')
log.info(f'\tMinimum time limit {mtl}')
log.info(f'\tUsed time limit {time_limit}')

2022-12-31 00:41:24,622 dwave.cloud.client.base INFO MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]


Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]


2022-12-31 00:41:24,636 dwave.cloud.client.base INFO MainThread Requested a solver that best matches feature filters={}


Requested a solver that best matches feature filters={}


2022-12-31 00:41:24,638 dwave.cloud.client.base INFO MainThread Fetching solvers according to filters={'supported_problem_types__contains': 'cqm'}, order_by='-properties.version'


...

Adding solver DQMSolver(id='hybrid_discrete_quadratic_model_version1')


2022-12-31 00:41:28,828 dwave.cloud.client.base INFO MainThread Adding solver CQMSolver(id='hybrid_constrained_quadratic_model_version1')


Adding solver CQMSolver(id='hybrid_constrained_quadratic_model_version1')


2022-12-31 00:41:28,914 dwave.cloud.client.base INFO MainThread Filtered solvers=[CQMSolver(id='hybrid_constrained_quadratic_model_version1')]


Filtered solvers=[CQMSolver(id='hybrid_constrained_quadratic_model_version1')]
Start sampling ... 
	Minimum time limit 18.71082593680027
	Used time limit 200


In [36]:
sampleset = sampler.sample_cqm(qm, time_limit=time_limit, label='ROADEF2020-QIO')
feasible_sampleset = sampleset.filter(lambda d: d.is_feasible)

log.info(f'Results')
log.info(f'\tSamples: {len(sampleset)}')
log.info(f'\tFeasible samples: {len(feasible_sampleset)}')

Results
	Samples: 103
	Feasible samples: 14


In [37]:
# Prepare result output

output = []
result = feasible_sampleset.first.sample if len(feasible_sampleset) > 0 else sampleset.first.sample
for i in range(I):
    for s in range(smax_i[i]+1):
        if result[f'x_{i}_{s}'] == 1 :
            output += [f'{intv[i]} {time[s]}']

if output_file != None :
    f = open(output_file, 'w')
    f.writelines([f'{o}\n' for o in output])
    f.close()
    log.info(f'Sampled solution wrote in {output_file}')
else:
    log.info(f'Sampled solution:')
    for o in output:
        log.info(o)


Sampled solution wrote in output.txt


In [38]:
# Check violated constraints

thold = 1e-5

log.info(f'Violations:')

violations = qm.violations(result)
for i,k in enumerate(violations):
    if violations[k] > thold : log.info(f'\t{i} - {k}')

Violations:


In [39]:
# Calculate the final average risk

riskfactors = []
for i in range(I):
    for s in range(smax_i[i]+1):
        r_i_s = 0
        for t in range(s,min(s+d_i_s[i][s]+1, T)):
            for r in range(R_t[t]):
                r_i_s += rk_r_i_t_s[r][i][t][s]/R_t[t]
        riskfactors += [r_i_s*result[f'x_{i}_{s}']]

log.info(f'Avarage risk: {round(sum(riskfactors)/T)}')


Avarage risk: 28644
